In [1]:
import os
import pandas as pd
import numpy as np
import rasterio
import gdal
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import mapping
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep

In [2]:
table = pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\Full_Data_Table.csv')
state_name = table['State'].unique()[0]

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5,16,19,23,24,25,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ds = rasterio.open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+state_name+'\\ID_NLCD_temp.tif')

In [4]:
ar =ds.read(1)

In [5]:
ar = ar.astype(str)

In [6]:
f = lambda x: x[:2]
vfunc = np.vectorize(f)
lulc = vfunc(ar)

y = lambda x: x[-6:]
yfunc = np.vectorize(y)
parcels = yfunc(ar)

In [7]:
#table['FFRC_LOCAL_ID'] = [(i+1) for i in list(table.index)]

In [8]:
cross = pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+state_name+'\\ID_Dictionary.csv')

In [9]:
df = table.merge(cross, on='PRCLDMPID', how = 'left')

In [10]:
parcels = parcels.astype(float)

In [11]:
id_df = df[['Value', 'Own_Type']]
id_dict = {0.0 : 0.0}
for i in range(len(id_df['Value'])):
    id_dict[id_df['Value'][i]] = id_df['Own_Type'][i]

In [12]:
test = list(id_df['Value'].unique())

In [13]:
m_dict = {}
for t in test:
    m_dict[t] = 0

In [14]:
missing = [i for i in np.unique(parcels) if i not in m_dict]

In [15]:
missing.remove(0.0)

In [16]:
for i in missing:
    id_dict[i] = .99999 

In [17]:
z = lambda x: id_dict[x]
zfunc = np.vectorize(z)
ownerships = zfunc(parcels)

In [18]:
ownerships = np.where(pd.isnull(ownerships) == True, 0.99999, ownerships)

In [19]:
ownerships = np.where((ownerships == 0.99999) | (ownerships == 0.0), ownerships, ownerships/100)

In [20]:
lulc = lulc.astype(float)

In [21]:
ownership_lands = ownerships+lulc

In [22]:
np.unique(ownership_lands)

array([10.     , 10.25   , 10.31   , 10.32   , 10.41   , 10.42   ,
       10.43   , 10.45   , 10.99999, 11.     , 11.25   , 11.31   ,
       11.32   , 11.41   , 11.42   , 11.43   , 11.45   , 11.99999,
       12.     , 12.25   , 12.31   , 12.32   , 12.41   , 12.42   ,
       12.43   , 12.45   , 12.99999, 13.     , 13.25   , 13.31   ,
       13.32   , 13.41   , 13.42   , 13.43   , 13.45   , 13.99999,
       14.     , 14.25   , 14.31   , 14.32   , 14.41   , 14.42   ,
       14.43   , 14.45   , 14.99999, 15.     , 15.25   , 15.31   ,
       15.32   , 15.41   , 15.42   , 15.43   , 15.45   , 15.99999,
       16.     , 16.25   , 16.31   , 16.32   , 16.41   , 16.42   ,
       16.43   , 16.45   , 16.99999, 20.     , 20.25   , 20.31   ,
       20.32   , 20.41   , 20.42   , 20.43   , 20.45   , 20.99999,
       30.     , 30.25   , 30.31   , 30.32   , 30.41   , 30.42   ,
       30.43   , 30.45   , 30.99999, 40.     , 40.25   , 40.31   ,
       40.32   , 40.41   , 40.42   , 40.43   , 40.45   , 40.99

In [23]:
def array_to_raster(array, reference, output_file):
    ref = gdal.Open(reference)
    srcgdal = np.array(ref.GetRasterBand(1).ReadAsArray())
    dst_filename = output_file
    x_pixels = srcgdal.shape[1]  # number of pixels in x
    y_pixels = srcgdal.shape[0]  # number of pixels in y
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
    dataset.SetGeoTransform(ref.GetGeoTransform())
    dataset.SetProjection(ref.GetProjection())    
    dataset.GetRasterBand(1).WriteArray(array)

In [24]:
#os.mkdir(os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\RASTERS\\'+state_name)

In [25]:
src = os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+state_name+'\\ID_NLCD_temp.tif'

In [26]:
array_to_raster(ownership_lands, src , os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\state_map.tif')

In [27]:
# Prettier plotting with seaborn
sns.set(font_scale=1.5)

# Get data and set working directory
#et.data.get_data("colorado-flood")
#os.chdir(os.path.join(et.io.HOME,
                      #'earth-analytics',
                      #'data'))

In [28]:
lidar_chm_im = rxr.open_rasterio(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\state_map.tif',
                                 masked=True).squeeze()

#f, ax = plt.subplots(figsize=(10, 5))
#lidar_chm_im.plot.imshow()
#ax.set(title="landcover vs ownership type")

#ax.set_axis_off()
#plt.show()

In [29]:
# Open crop extent (your study area extent boundary)
crop_extent = gpd.read_file(os.path.dirname(os.getcwd())+'\\INPUTS\\State_Boundaries.shp')
state_fips = pd.read_csv(os.path.dirname(os.getcwd())+'\\INPUTS\\State_Fips_Dictionary.csv')
state_fips = state_fips.rename(columns={'State Abbreviation' : 'State_Abbreviation'})
sfd = state_fips.set_index('FIPS Code').State_Abbreviation.to_dict()
crop_extent['STATEFP'] = crop_extent['STATEFP'].astype(int)
crop_extent['State_Abbreviation'] = crop_extent.STATEFP.map(sfd)
crop_extent =crop_extent.loc[crop_extent['State_Abbreviation'] == state_name]

In [30]:
#fig, ax = plt.subplots(figsize=(6, 6))

#crop_extent.plot(ax=ax)

#ax.set_title("Shapefile Crop Extent",fontsize=16)
#plt.show()

In [31]:
#f, ax = plt.subplots(figsize=(10, 5))
#lidar_chm_im.plot.imshow(ax=ax)

#crop_extent.plot(ax=ax,alpha=.8)
#ax.set(title="Raster Layer with Shapefile Overlayed")

#ax.set_axis_off()
#plt.show()

In [32]:
lidar_clipped = lidar_chm_im.rio.clip(crop_extent.geometry.apply(mapping),
                                      # This is needed if your GDF is in a diff CRS than the raster data
                                      crop_extent.crs)

#f, ax = plt.subplots(figsize=(10, 4))
#lidar_clipped.plot(ax=ax)
#ax.set(title="Raster Layer Cropped to Geodataframe Extent")
#ax.set_axis_off()
#plt.show()

In [33]:
path_to_tif_file = os.path.dirname(os.getcwd())+'\\OUTPUTS\\FINAL\\RASTERS\\'+state_name+'\\'+ state_name+'_LULC_OWNERSHIP.tif'

# Write the data to a new geotiff file
lidar_clipped.rio.to_raster(path_to_tif_file)